**Enrolment Data Preprocessing and Maintaining the Consistency of State Names throughout the Dataset**

In [ ]:
import pandas as pd

# 1. List and Merge the 3 Enrolment Files
enrol_files = [
    'api_data_aadhar_enrolment_0_500000.csv',
    'api_data_aadhar_enrolment_500000_1000000.csv',
    'api_data_aadhar_enrolment_1000000_1006029.csv'
]
df_enrol = pd.concat([pd.read_csv(f) for f in enrol_files], ignore_index=True)

# 2. Standardize Casing
df_enrol['state'] = df_enrol['state'].astype(str).str.strip().str.title()

# 3. Apply Master Mapping (Fixes duplicates, merges UTs, maps cities to states)
mapping = {
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',
    'Andman & Nicobar': 'Andaman and Nicobar Islands',
    'Andaman And Nicobar Islands': 'Andaman and Nicobar Islands',
    'Jammu & Kashmir': 'Jammu and Kashmir',
    'Jammu And Kashmir': 'Jammu and Kashmir',
    'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman & Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra And Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman And Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dnh And Dd': 'Dadra and Nagar Haveli and Daman and Diu',
    'Chhatisgarh': 'Chhattisgarh',
    'Orissa': 'Odisha',
    'Odissa': 'Odisha',
    'Pondicherry': 'Puducherry',
    'West Bangal': 'West Bengal',
    'West Bengli': 'West Bengal',
    'Westbengal': 'West Bengal',
    'West  Bengal': 'West Bengal',
    'Uttaranchal': 'Uttarakhand',
    'Nagpur': 'Maharashtra',
    'Jaipur': 'Rajasthan',
    'Darbhanga': 'Bihar',
    'Madanapalle': 'Andhra Pradesh',
    'Puttenahalli': 'Karnataka',
    'Raja Annamalai Puram': 'Tamil Nadu',
    'Balanagar': 'Telangana',
    'Gorakhpur': 'Uttar Pradesh',
    'Rajkot': 'Gujarat',
    'Tumakuru': 'Karnataka'
}
df_enrol['state'] = df_enrol['state'].replace(mapping)

# 4. Strict Filter for the 36 Official States/UTs
final_valid_list = [
    'Andaman and Nicobar Islands', 'Andhra Pradesh', 'Arunachal Pradesh',
    'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh',
    'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Goa', 'Gujarat',
    'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand',
    'Karnataka', 'Kerala', 'Ladakh', 'Lakshadweep', 'Madhya Pradesh',
    'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha',
    'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu',
    'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'
]
df_enrol_final = df_enrol[df_enrol['state'].isin(final_valid_list)].copy()

# Save to the final ready-to-use CSV
df_enrol_final.to_csv('cleaned_enrolment_ready.csv', index=False)

In [ ]:
import pandas as pd

# Load the cleaned enrolment dataset
df_enrol = pd.read_csv('cleaned_enrolment_ready.csv')

# Get the list of unique states and sort them alphabetically
unique_states = sorted(df_enrol['state'].unique())

# Print the states vertically
print("--- Unique States in Enrolment Data ---")
for state in unique_states:
    print(state)

# Print the total count of unique states
print(f"\nTotal Unique States: {len(unique_states)}")

# Verify if there are any unexpected values or missing rows
print(f"Total Rows in Dataset: {len(df_enrol):,}")

--- Unique States in Enrolment Data ---
Andaman and Nicobar Islands
Andhra Pradesh
Arunachal Pradesh
Assam
Bihar
Chandigarh
Chhattisgarh
Dadra and Nagar Haveli and Daman and Diu
Delhi
Goa
Gujarat
Haryana
Himachal Pradesh
Jammu and Kashmir
Jharkhand
Karnataka
Kerala
Ladakh
Lakshadweep
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Nagaland
Odisha
Puducherry
Punjab
Rajasthan
Sikkim
Tamil Nadu
Telangana
Tripura
Uttar Pradesh
Uttarakhand
West Bengal

Total Unique States: 36
Total Rows in Dataset: 1,005,889


In [ ]:
import pandas as pd

# Load our two "Gold Standard" cleaned files
df_demo = pd.read_csv('cleaned_demographic_ready.csv')
df_enrol = pd.read_csv('cleaned_enrolment_ready.csv')

# 1. Aggregate Demographic Updates by State
state_updates = df_demo.groupby('state')[['demo_age_5_17', 'demo_age_17_']].sum()
state_updates['total_updates'] = state_updates.sum(axis=1)

# 2. Aggregate New Enrolments by State
state_enrol = df_enrol.groupby('state')[['age_0_5', 'age_5_17', 'age_18_greater']].sum()
state_enrol['total_enrolments'] = state_enrol.sum(axis=1)

# 3. Merge them into one Analysis Table
analysis_df = pd.merge(state_updates, state_enrol, on='state', suffixes=('_upd', '_enr'))

# 4. Create an Innovation Metric: "Update-to-Enrolment Ratio"
# This tells us if a state is in "Maintenance mode" (updates) or "Growth mode" (new users)
analysis_df['service_ratio'] = analysis_df['total_updates'] / analysis_df['total_enrolments']

print(analysis_df[['total_updates', 'total_enrolments', 'service_ratio']].sort_values(by='total_enrolments', ascending=False))

                   total_updates  total_enrolments  service_ratio
state                                                            
Uttar Pradesh           854890.0           1018629       0.839256
Bihar                   513431.0            609585       0.842263
Madhya Pradesh          368855.0            493970       0.746715
West Bengal             403828.0            375340       1.075899
Maharashtra             323234.0            369139       0.875643
Rajasthan               339938.0            348458       0.975549
Gujarat                 248530.0            280549       0.885870
Assam                    95548.0            230197       0.415071
Karnataka               199593.0            223235       0.894094
Tamil Nadu              199656.0            220789       0.904284
Jharkhand               158267.0            157539       1.004621
Telangana               176499.0            131574       1.341443
Andhra Pradesh          233762.0            127686       1.830757
Odisha    